In [51]:
import numpy as np
import pandas as pd

In [52]:
bowler = pd.read_excel('6671501a18c55_round2_input.xlsx', sheet_name='bowler_scorecard')

In [53]:
bowler.head()

,match id,bowler,bowler_id,bowler_dob,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt,ingestion_timestamp
0,8587795,PM Se,55299,1984-11-25,AUS:Right-hand bat:Right-arm fast-medium:,0,0,2,38,0,24,9.50,0,8,4,1,1,0,2021-01-08,2022-03-21 03:44:58
1,8587795,Md Ni,181404,1985-01-01,AFG:Right-hand bat:Right-arm offbreak:,0,0,1,9,0,6,9.00,0,1,1,0,0,0,2021-01-08,2022-03-21 03:44:58
2,8587795,Id Wm,1594319,1988-12-18,PAK:Left-hand bat:Slow left-arm orthodox:,0,0,1,28,1,24,7.00,0,6,3,0,0,0,2021-01-08,2022-03-21 03:44:58
3,8587795,KW Rn,1905847,1991-02-12,AUS:Right-hand bat:Right-arm fast-medium:,0,0,1,40,1,24,10.00,0,10,2,3,3,0,2021-01-08,2022-03-21 03:44:58
4,8587795,DR Bs,2170762,1991-04-30,ENG:Right-hand bat:Slow left-arm orthodox:,0,0,2,37,1,24,9.25,0,7,4,1,0,0,2021-01-08,2022-03-21 03:44:58


In [54]:
# List of columns to keep
columns_to_keep = [
    'bowler',                   
    'bowler_id',  
    'bowler_details',              
    'runs',         
    'wicket_count',                      
    'balls_bowled',                         
    'economy',     
    'match_dt'          
]

# Keep only the specified columns and discard the rest
bowler = bowler[columns_to_keep]

In [55]:
bowler.head()

,bowler,bowler_id,bowler_details,runs,wicket_count,balls_bowled,economy,match_dt
0,PM Se,55299,AUS:Right-hand bat:Right-arm fast-medium:,38,0,24,9.50,2021-01-08
1,Md Ni,181404,AFG:Right-hand bat:Right-arm offbreak:,9,0,6,9.00,2021-01-08
2,Id Wm,1594319,PAK:Left-hand bat:Slow left-arm orthodox:,28,1,24,7.00,2021-01-08
3,KW Rn,1905847,AUS:Right-hand bat:Right-arm fast-medium:,40,1,24,10.00,2021-01-08
4,DR Bs,2170762,ENG:Right-hand bat:Slow left-arm orthodox:,37,1,24,9.25,2021-01-08


In [56]:
bowler=bowler.sort_values(by='bowler_id')

In [57]:
bowler.head()

,bowler,bowler_id,bowler_details,runs,wicket_count,balls_bowled,economy,match_dt
5230,DT Cn,34061,AUS:Right-hand bat:Right-arm fast-medium:,23,0,6,23.00,2021-07-14
9590,DT Cn,34061,AUS:Right-hand bat:Right-arm fast-medium:,59,0,18,19.67,2022-06-17
446,DT Cn,34061,AUS:Right-hand bat:Right-arm fast-medium:,9,0,6,9.00,2021-01-30
4736,DT Cn,34061,AUS:Right-hand bat:Right-arm fast-medium:,21,0,12,10.50,2021-09-26
5548,DT Cn,34061,AUS:Right-hand bat:Right-arm fast-medium:,8,3,18,2.67,2021-12-21


In [58]:
# Calculate Strike Rate for each match
bowler['Strike_Rate'] = np.where(
    bowler['wicket_count'] > 0,
    bowler['balls_bowled'] / bowler['wicket_count'],
    np.inf  # or np.nan to indicate no wickets
)

# Replace infinite values with NaN for mean calculation
bowler['Strike_Rate'].replace(np.inf, np.nan, inplace=True)

# Calculate the mean Strike Rate for each bowler
mean_strike_rate = bowler.groupby('bowler_id')['Strike_Rate'].transform('mean')

# Add the mean Strike Rate as a new column
bowler['Bowling_Strike_Rate'] = mean_strike_rate

In [59]:
# Calculate Average Rate for each match
bowler['Average'] = np.where(
    bowler['wicket_count'] > 0,
    bowler['runs'] / bowler['wicket_count'],
    np.inf  # or np.nan to indicate no wickets
)

# Replace infinite values with NaN for mean calculation
bowler['Average'].replace(np.inf, np.nan, inplace=True)

# Calculate the mean Average Rate for each bowler
mean_average_rate = bowler.groupby('bowler_id')['Average'].transform('mean')

# Add the mean Average Rate as a new column
bowler['Bowling_Average'] = mean_average_rate

In [60]:
bowler.rename(columns={'economy':'economy_per_match'},inplace=True)

In [61]:
def calculate_bowler_stats(group):
    Wickets = group['wicket_count'].sum()
    Economy = group['economy_per_match'].mean()
    W_Per_Innings = (group['wicket_count'] >= 4).sum()
    return pd.Series({
        'Wickets': Wickets,
        'Economy': Economy,
        '4W_Per_Innings':W_Per_Innings,
        
    })

# Group by 'bowler_id' and apply the calculation function
bowler_stats = bowler.groupby('bowler_id').apply(calculate_bowler_stats).reset_index()

# Merge the calculated stats back into the original DataFrame
bowler = bowler.merge(bowler_stats, on='bowler_id', how='left')

In [62]:
# Define the point system based on the given ranges
def assign_strike_rate_points(sr):
    if sr <= 15:
        return 30
    elif 15 < sr <= 19:
        return 20
    elif 19 < sr <= 24:
        return 10
    else:
        return 0

def assign_economy_points(econ):
    if econ <= 3:
        return 50
    elif 3 < econ <= 5:
        return 40
    elif 5 < econ < 7:
        return 30
    else:
        return 0

def assign_avg_points(avg):
    if avg <= 20:
        return 30
    elif 20 < avg <= 30:
        return 20
    elif 30 < avg <= 40:
        return 10
    else:
        return 0
    
def assign_wickets_points(w):
    if w >= 4:
        return 30
    elif w>=2:
        return 20
    elif w>=1:
        return 10
    else:
        return 0

# Apply the point system to the respective columns
bowler['Strike_Rate_Points'] = bowler['Bowling_Strike_Rate'].apply(assign_strike_rate_points)
bowler['Overall_Economy_Points'] = bowler['Economy'].apply(assign_economy_points)
bowler['Avg_Points'] = bowler['Bowling_Average'].apply(assign_avg_points)
bowler['Wicket_Points'] = bowler['4W_Per_Innings'].apply(assign_wickets_points)




In [63]:
bowler['Total_Points'] = bowler['Strike_Rate_Points'] + bowler['Overall_Economy_Points'] + bowler['Avg_Points'] + bowler['Wicket_Points']

In [64]:
# Converting 'match_dt' to datetime for proper sorting
bowler['match_dt'] = pd.to_datetime(bowler['match_dt'])

# Sorting by 'Total_Points' (descending) and 'match_dt' (ascending) where 'bowler_id' is equal
bowler.sort_values(by=['Total_Points', 'bowler_id', 'match_dt'],
                    ascending=[False, True, True], inplace=True)

In [66]:
import pandas as pd
import numpy as np
from scipy import stats
iowwoebdc
# Assuming the dataframe is already loaded as 'df'

def calculate_rankings(df):
    # 1. Handle np.inf values
    df['Economy'] = df['Economy'].replace([np.inf, -np.inf], np.nan)
    df['Bowling_Strike_Rate'] = df['Bowling_Strike_Rate'].replace([np.inf, -np.inf], np.nan)
    df['Bowling_Average'] = df['Bowling_Average'].replace([np.inf, -np.inf], np.nan)
    
    # Fill NaN values with the mean of the column
    df['Economy'] = df['Economy'].fillna(df['Economy'].mean())
    df['Bowling_Strike_Rate'] = df['Bowling_Strike_Rate'].fillna(df['Bowling_Strike_Rate'].mean())
    df['Bowling_Average'] = df['Bowling_Average'].fillna(df['Bowling_Average'].mean())

    # 2. Group by bowler_id
    grouped = df.groupby('bowler_id')

    # 3. Calculate metrics for each bowler
    bowler_stats = grouped.agg({
        'Economy': ['mean', 'std'],
        'Bowling_Strike_Rate': ['mean', 'std'],
        'Bowling_Average': ['mean', 'std'],
        'Wickets': 'sum',
        'match_dt': 'max',  # Most recent match date
        'bowler_id': 'count'  # Number of matches played
    })

    # Flatten column names
    bowler_stats.columns = ['_'.join(col).strip() for col in bowler_stats.columns.values]

    # 4. Calculate consistency score (lower is better)
    bowler_stats['consistency_score'] = (
        stats.zscore(bowler_stats['Economy_std']) +
        stats.zscore(bowler_stats['Bowling_Strike_Rate_std']) +
        stats.zscore(bowler_stats['Bowling_Average_std'])
    )

    # 5. Calculate recency score
    bowler_stats['recency_score'] = (pd.to_datetime(bowler_stats['match_dt_max']) - pd.to_datetime(bowler_stats['match_dt_max']).min()).dt.days

    # 6. Calculate performance score (lower is better)
    bowler_stats['performance_score'] = (
        stats.zscore(bowler_stats['Economy_mean']) +
        stats.zscore(bowler_stats['Bowling_Strike_Rate_mean']) +
        stats.zscore(bowler_stats['Bowling_Average_mean']) -
        stats.zscore(bowler_stats['Wickets_sum'])
    )

    # 7. Adjust scores based on number of matches played
    matches_weight = np.log1p(bowler_stats['bowler_id_count']) / np.log1p(bowler_stats['bowler_id_count'].max())
    
    bowler_stats['consistency_score'] *= (1 - matches_weight)
    bowler_stats['performance_score'] *= (1 - matches_weight)

    # 8. Calculate final ranking score (lower is better)
    bowler_stats['final_score'] = (
        bowler_stats['consistency_score'] * 0.3 +
        bowler_stats['performance_score'] * 0.5 -
        stats.zscore(bowler_stats['recency_score']) * 0.2
    )

    # 9. Rank bowlers
    bowler_stats['rank'] = bowler_stats['final_score'].rank()

    return bowler_stats.sort_values('rank')

# Calculate rankings
rankings = calculate_rankings(bowler)

# Display top 10 bowlers
print(rankings.head(10))

           Economy_mean  Economy_std  Bowling_Strike_Rate_mean  \
bowler_id                                                        
34061          9.490000          0.0                 13.050000   
41740          5.900000          0.0                  6.666667   
49496          6.203043          0.0                 16.156250   
55299          8.097429          0.0                 14.453333   
62432          7.826494          0.0                 13.156250   
63307          6.832727          0.0                 18.000000   
65183          9.090000          0.0                 12.625000   
74087          8.110000          0.0                 14.150000   
74367          8.254286          0.0                 15.150000   
82081          7.750000          0.0                 15.133084   

           Bowling_Strike_Rate_std  Bowling_Average_mean  Bowling_Average_std  \
bowler_id                                                                       
34061                          0.0           

In [67]:
rankings.head(20)

,Economy_mean,Economy_std,Bowling_Strike_Rate_mean,Bowling_Strike_Rate_std,Bowling_Average_mean,Bowling_Average_std,Wickets_sum,match_dt_max,bowler_id_count,consistency_score,recency_score,performance_score,final_score,rank
bowler_id,,,,,,,,,,,,,,
34061,9.490000,0.0,13.050000,0.0,17.908333,0.0,1242.0,2023-01-06,46,NaN,735,-0.200032,NaN,NaN
41740,5.900000,0.0,6.666667,0.0,5.000000,0.0,50.0,2023-12-30,10,NaN,1093,-2.637845,NaN,NaN
49496,6.203043,0.0,16.156250,0.0,16.104167,0.0,713.0,2023-12-26,23,NaN,1089,-0.459245,NaN,NaN
55299,8.097429,0.0,14.453333,0.0,18.540000,0.0,1610.0,2023-12-29,35,NaN,1092,-0.343803,NaN,NaN
62432,7.826494,0.0,13.156250,0.0,15.940972,0.0,5544.0,2023-12-21,77,NaN,1084,-0.523591,NaN,NaN
63307,6.832727,0.0,18.000000,0.0,20.750000,0.0,88.0,2021-07-16,11,NaN,196,0.291559,NaN,NaN
65183,9.090000,0.0,12.625000,0.0,19.458333,0.0,144.0,2021-08-27,9,NaN,238,-0.094518,NaN,NaN
74087,8.110000,0.0,14.150000,0.0,18.737500,0.0,1260.0,2023-08-18,42,NaN,959,-0.247678,NaN,NaN
74367,8.254286,0.0,15.150000,0.0,18.116667,0.0,210.0,2023-07-07,14,NaN,917,-0.045292,NaN,NaN


In [ ]:
# Convert 'match_dt' to datetime
bowler['match_dt'] = pd.to_datetime(bowler['match_dt'])

# Replace np.inf with a high value
for metric in ['Economy', 'Bowling_Strike_Rate', 'Bowling_Average']:
    finite_values = bowler[metric].replace(np.inf, np.nan)
    high_value = finite_values.max() * 10  # Use a high finite value
    bowler[metric].replace(np.inf, high_value, inplace=True)

# Normalize the metrics
metrics = ['Economy', 'Bowling_Strike_Rate', 'Bowling_Average', 'Wickets']
bowler_normalized = bowler[metrics].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Calculate Cumulative Score
weights = {'Economy': 0.25, 'Bowling_Strike_Rate': 0.25, 'Bowling_Average': 0.25, 'Wickets': 0.25}
bowler['Cumulative_Score'] = sum(weights[metric] * bowler_normalized[metric] for metric in metrics)

# Measure Consistency (Standard Deviation)
bowler['Consistency_Score'] = 1 / (1 + bowler_normalized.std(axis=1))

# Incorporate Recency
current_time = pd.Timestamp('now')
lambda_decay = 0.01
bowler['Recency_Score'] = bowler.apply(
    lambda row: np.exp(-lambda_decay * (current_time - row['match_dt']).days), axis=1
)
bowler['Recency_Score'] = bowler['Recency_Score'] / bowler['Recency_Score'].sum()

# Volume-based adjustment
min_balls_threshold = 30
bowler['Volume_Adjustment'] = bowler['balls_bowled'] / bowler['balls_bowled'].max()
bowler['Volume_Adjustment'] = bowler['Volume_Adjustment'].apply(lambda x: x if x > (min_balls_threshold / bowler['balls_bowled'].max()) else 0.1)

# Adjust final score with volume-based penalty
final_weights = {'Cumulative_Score': 0.5, 'Consistency_Score': 0.25, 'Recency_Score': 0.25}
bowler['Final_Score'] = (
    final_weights['Cumulative_Score'] * bowler['Cumulative_Score'] +
    final_weights['Consistency_Score'] * bowler['Consistency_Score'] +
    final_weights['Recency_Score'] * bowler['Recency_Score']
) * bowler['Volume_Adjustment']

# Rank the bowlers
bowler['Rank'] = bowler['Final_Score'].rank(ascending=False)

In [ ]:
bowler.sort_values(by='Rank',ascending=False,inplace=True)

In [ ]:
final_output = './Files/bowlerRanks2.0.xlsx'
bowler.to_excel(final_output, index=False)

In [ ]:
bowler.drop_duplicates(subset='bowler_id',keep='first',inplace=True)

In [ ]:
final_output = './Files/bowlerRanks3.0.xlsx'
bowler.to_excel(final_output, index=False)

In [ ]:
# Convert 'match_dt' to datetime
bowler['match_dt'] = pd.to_datetime(bowler['match_dt'])

# Drop duplicates based on 'bowler_id', keeping the first occurrence
bowler.drop_duplicates(subset='bowler_id', keep='first', inplace=True)

# Replace np.inf with a high value
for metric in ['Economy', 'Bowling_Strike_Rate', 'Bowling_Average']:
    finite_values = bowler[metric].replace(np.inf, np.nan)
    high_value = finite_values.max() * 10  # Use a high finite value
    bowler[metric].replace(np.inf, high_value, inplace=True)

# Normalize the metrics
metrics = ['Economy', 'Bowling_Strike_Rate', 'Bowling_Average', 'Wickets']
bowler_normalized = bowler[metrics].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Calculate Cumulative Score
weights = {'Economy': 0.25, 'Bowling_Strike_Rate': 0.25, 'Bowling_Average': 0.25, 'Wickets': 0.25}
bowler['Cumulative_Score'] = sum(weights[metric] * bowler_normalized[metric] for metric in metrics)

# Measure Consistency (Standard Deviation)
bowler['Consistency_Score'] = 1 / (1 + bowler_normalized.std(axis=1))

# Incorporate Recency
current_time = pd.Timestamp('now')
lambda_decay = 0.01
bowler['Recency_Score'] = bowler.apply(
    lambda row: np.exp(-lambda_decay * (current_time - row['match_dt']).days), axis=1
)
# Normalize Recency Score
bowler['Recency_Score'] = (bowler['Recency_Score'] - bowler['Recency_Score'].min()) / (bowler['Recency_Score'].max() - bowler['Recency_Score'].min())

# Calculate occurrences
occurrences = bowler.groupby('bowler_id').size().reset_index(name='Occurrences')
bowler = bowler.merge(occurrences, on='bowler_id', how='left')

# Normalize Occurrences
bowler['Occurrences_Normalized'] = (bowler['Occurrences'] - bowler['Occurrences'].min()) / (bowler['Occurrences'].max() - bowler['Occurrences'].min())

# Volume-based adjustment
min_balls_threshold = 30
bowler['Volume_Adjustment'] = bowler['balls_bowled'] / bowler['balls_bowled'].max()
bowler['Volume_Adjustment'] = bowler['Volume_Adjustment'].apply(lambda x: x if x > (min_balls_threshold / bowler['balls_bowled'].max()) else 0.1)

# Adjust final score with volume-based penalty and occurrences
final_weights = {'Cumulative_Score': 0.4, 'Consistency_Score': 0.2, 'Recency_Score': 0.2, 'Occurrences_Normalized': 0.2}
bowler['Final_Score'] = (
    final_weights['Cumulative_Score'] * bowler['Cumulative_Score'] +
    final_weights['Consistency_Score'] * bowler['Consistency_Score'] +
    final_weights['Recency_Score'] * bowler['Recency_Score'] +
    final_weights['Occurrences_Normalized'] * bowler['Occurrences_Normalized']
) * bowler['Volume_Adjustment']

# Rank the bowlers
bowler['Rank'] = bowler['Final_Score'].rank(ascending=False)

# Display the results
df_result = bowler[['bowler', 'bowler_id', 'Final_Score', 'Rank', 'Occurrences']]
print(df_result)


        bowler  bowler_id  Final_Score  Rank  Occurrences
0     DB Ri Ta    1663920          NaN   NaN            1
1        Ht Sh    5636658          NaN   NaN            1
2         W Ck    8354800          NaN   NaN            1
3        SJ Mi    8934764          NaN   NaN            1
4         H Me    9362702          NaN   NaN            1
...        ...        ...          ...   ...          ...
1506     Nh Lf    9804927          NaN   NaN            1
1507      Y Sa    9822231          NaN   NaN            1
1508      T Ti    9822245          NaN   NaN            1
1509     SK Ds    9822273          NaN   NaN            1
1510     Sv Dy    9822322          NaN   NaN            1

[1511 rows x 5 columns]


In [ ]:
final_output = './Files/bowlerRanks4.0(NumberOfOccurence).xlsx'
bowler.to_excel(final_output, index=False)